<a href="https://colab.research.google.com/github/bhavya-kashyap/AMC_crosswalk/blob/main/abstract_pubmed_webscrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt update
!apt install chromium-chromedriver
# !pip install selenium
# !pip install bs4
!pip install pymed

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http

In [2]:
import pandas as pd
import numpy as np
from pymed import PubMed
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.by import By
# from bs4 import BeautifulSoup
import time
import math
from google.colab import files

In [ ]:
# Data parameters
# https://www.webofscience.com/wos/woscc/summary/f05a3dd0-ac52-4364-859d-aea68cf4f4d3-583fd5ef/relevance/1
# SO = (MMWR RECOMMENDATIONS AND REPORTS OR NEW ENGLAND JOURNAL OF MEDICINE OR NATURE MEDICINE OR NATURE REVIEWS IMMUNOLOGY OR MORBIDITY AND MORTALITY WEEKLY REPORT OR LANCET OR PHYSIOLOGICAL REVIEWS OR NATURE REVIEWS DRUG DISCOVERY OR ANNUAL REVIEW OF PATHOLOGY: MECHANISMS OF DISEASE OR JOURNAL OF CLINICAL ONCOLOGY OR LANCET GLOBAL HEALTH OR ANNALS OF ONCOLOGY OR ANNUAL REVIEW OF PUBLIC HEALTH OR NATURE REVIEWS DISEASE PRIMERS OR ACCOUNTS OF CHEMICAL RESEARCH OR ANNUAL REVIEW OF CLINICAL PSYCHOLOGY OR JOURNAL OF EXPERIMENTAL MEDICINE OR MOLECULAR SYSTEMS BIOLOGY OR ANNUAL REVIEW OF MICROBIOLOGY OR JAMA - JOURNAL OF THE AMERICAN MEDICAL ASSOCIATION OR LANCET DIGITAL HEALTH OR REPORTS ON PROGRESS IN PHYSICS OR NATURE BIOMEDICAL ENGINEERING OR EMBO JOURNAL OR JAMA PSYCHIATRY OR NATURE REVIEWS MOLECULAR CELL BIOLOGY OR CELL OR NATURE REVIEWS GENETICS OR NATURE BIOTECHNOLOGY OR NATURE REVIEWS CANCER OR NATURE GENETICS OR ANNUAL REVIEW OF BIOCHEMISTRY OR NATURE CATALYSIS OR NATURE METHODS OR CANCER CELL OR MOLECULAR CELL OR NATURE CELL BIOLOGY OR LANCET DIABETES AND ENDOCRINOLOGY OR NATURE MICROBIOLOGY OR ANNUAL REVIEW OF PLANT BIOLOGY OR ANNUAL REVIEW OF GENETICS OR CELL HOST AND MICROBE OR GENOME BIOLOGY OR CELL METABOLISM OR NATURE REVIEWS ENDOCRINOLOGY OR NATURE STRUCTURAL AND MOLECULAR BIOLOGY OR NUCLEIC ACIDS RESEARCH OR PHARMACOLOGICAL REVIEWS OR ANNUAL REVIEW OF PHARMACOLOGY AND TOXICOLOGY OR NANO TODAY OR DRUG RESISTANCE UPDATES OR JOURNAL FOR IMMUNOTHERAPY OF CANCER OR TRENDS IN PHARMACOLOGICAL SCIENCES OR EMERGING MICROBES AND INFECTIONS OR PHARMACOLOGY AND THERAPEUTICS OR MOLECULAR THERAPY OR NPJ VACCINES OR ADVANCED DRUG DELIVERY REVIEWS OR NATURAL PRODUCT REPORTS OR CELL CHEMICAL BIOLOGY OR PROTEIN AND CELL OR CELLULAR AND MOLECULAR LIFE SCIENCES OR MEDICINAL RESEARCH REVIEWS OR NEUROPSYCHOPHARMACOLOGY OR ANTIVIRAL RESEARCH OR ACTA PHARMACEUTICA SINICA B OR PAIN OR THERANOSTICS OR ADVANCED HEALTHCARE MATERIALS OR BRITISH JOURNAL OF PHARMACOLOGY OR ENVIRONMENTAL POLLUTION OR ALZHEIMER'S AND DEMENTIA OR HEALTH AFFAIRS OR LANCET PLANETARY HEALTH OR IMPLEMENTATION SCIENCE OR INFECTIOUS DISEASE MODELLING OR BMJ GLOBAL HEALTH OR GLOBALIZATION AND HEALTH OR JOURNAL OF HEALTH ECONOMICS OR JOURNAL OF THE AMERICAN MEDICAL DIRECTORS ASSOCIATION OR MILBANK QUARTERLY OR EUROPEAN HEART JOURNAL. QUALITY OF CARE &AMP, CLINICAL OUTCOMES OR BMJ QUALITY AND SAFETY OR HEALTH SERVICES RESEARCH OR MEDICAL CARE RESEARCH AND REVIEW OR VALUE IN HEALTH OR POLICY AND INTERNET OR INTERNATIONAL JOURNAL OF DRUG POLICY OR INTERNATIONAL JOURNAL FOR EQUITY IN HEALTH OR JOURNAL OF GLOBAL HEALTH OR PHARMACOECONOMICS OR HEALTH RESEARCH POLICY AND SYSTEMS OR GLOBAL HEALTH RESEARCH AND POLICY OR INTERNATIONAL JOURNAL OF TRANSGENDER HEALTH OR JCO ONCOLOGY PRACTICE OR PREVENTING CHRONIC DISEASE OR NATURE OR SCIENCE OR SCIENCE ADVANCES OR PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES OF THE UNITED STATES OF AMERICA OR NATIONAL SCIENCE REVIEW OR SCIENCE BULLETIN OR RESEARCH OR JOURNAL OF ADVANCED RESEARCH OR ISCIENCE OR SCIENTIFIC REPORTS OR TSINGHUA SCIENCE AND TECHNOLOGY OR PROCEEDINGS OF THE JAPAN ACADEMY SERIES B: PHYSICAL AND BIOLOGICAL SCIENCES OR ADVANCED THEORY AND SIMULATIONS OR PROGRESS IN NATURAL SCIENCE: MATERIALS INTERNATIONAL OR PLOS ONE OR SCIENCE AND TECHNOLOGY STUDIES OR ROYAL SOCIETY OPEN SCIENCE OR RESEARCH IN SCIENCE AND TECHNOLOGICAL EDUCATION OR INTERNATIONAL JOURNAL OF BIFURCATION AND CHAOS IN APPLIED SCIENCES AND ENGINEERING OR LATIN AMERICAN RESEARCH REVIEW OR FRACTALS OR FACETS OR HELIYON OR JOURNAL OF KING SAUD UNIVERSITY - SCIENCE OR ARABIAN JOURNAL FOR SCIENCE AND ENGINEERING OR CLINICAL AND TRANSLATIONAL IMMUNOLOGY OR JOURNAL OF PAIN AND SYMPTOM MANAGEMENT OR INTERNATIONAL JOURNAL OF NURSING STUDIES OR NURSE EDUCATION TODAY OR JOURNAL OF NURSING SCHOLARSHIP OR JOURNAL OF PALLIATIVE MEDICINE OR PATIENT OR CLINICAL SIMULATION IN NURSING OR INTERNATIONAL NURSING REVIEW OR JOURNAL OF CLINICAL NURSING OR WORLDVIEWS ON EVIDENCE-BASED NURSING OR NURSE EDUCATION IN PRACTICE OR JOURNAL OF ADVANCED NURSING OR JOURNAL OF SCHOOL NURSING OR JOURNAL OF RELIGION AND HEALTH OR INTERNATIONAL JOURNAL OF NURSING SCIENCES OR NURSING OUTLOOK OR PALLIATIVE AND SUPPORTIVE CARE OR SYMBOLIC INTERACTION OR JOURNAL OF PROFESSIONAL NURSING OR JOURNAL OF TRANSCULTURAL NURSING OR JOURNAL OF NURSING EDUCATION OR BMC NURSING OR JOURNAL OF MULTIDISCIPLINARY HEALTHCARE)
# Publication years: last 5 years (2018 - 2022)
# Affiliations: Emory university
# Size of data: 1000 records
# primary focus is to get the pubmed ID

In [3]:
# df = pd.read_csv("Georgia_pubmed.csv", encoding='cp1252')
# df.shape

In [8]:
# df.head()
# df.drop(columns = ['Publication Type','Authors','Author Full Names', 'Group Authors','Source Title','Document Type', 'Publication Date','Publication Year', 'Issue','Start Page','End Page', 'Article Number','DOI','Volume', 'DOI Link', 'Early Access Date', 'UT (Unique WOS ID)', 'Web of Science Record'], inplace=True)

In [7]:
# baseurl = 'https://pubmed.ncbi.nlm.nih.gov/'
# urls = []
# for i in range(len(df)):
#     urls.append(baseurl + str(df.loc[i]['PMID'].astype(int)) + '/')
# df['URL'] = urls
# df.head(10)

In [ ]:
# sample = df.head(20)

In [ ]:
# def driversetup():
#       options = webdriver.ChromeOptions()
#       #run Selenium in headless mode
#       options.add_argument('--headless')
#       options.add_argument('--no-sandbox')
#       options.add_argument("enable-automation")
#       #overcome limited resource problems
#       options.add_argument('--disable-dev-shm-usage')
#       options.add_argument("lang=en")
#       #open Browser in maximized mode
#       options.add_argument("start-maximized")
#       #disable infobars
#       options.add_argument("disable-infobars")
#       #disable extension
#       options.add_argument("--disable-browser-side-navigation")
#       options.add_argument("--disable-gpu")
#       options.add_argument("--disable-extensions")
#       options.add_argument("--incognito")
#       options.add_argument("--disable-blink-features=AutomationControlled")
#       driver = webdriver.Chrome(options=options)
#       driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")
#       return driver

# def pagesource(url, driver):
#       driver = driver
#       driver.get(url)
#       time.sleep(5)
#       soup = BeautifulSoup(driver.page_source)
#       return soup

In [10]:
# d = driversetup()
# abstracts = []
# for i in range(len(df)):
#   s = pagesource(df.loc[i]['URL'], d)
#   abstract = list(s.findAll('div', class_="abstract-content selected"))
#   a = ''
#   for j in range(len(abstract)):
#     a = a + ' ' + abstract[j].text
#   abstracts.append(a)
#   print(i, a)

# df["PubMedAbstracts"] = abstracts

# df.to_csv('GeorgiaPubmedAbstracts.csv', index=False)
# files.download("GeorgiaPubmedAbstracts.csv")
# abs = pd.DataFrame({'Abstracts' : abstracts})
# abs.to_csv('GeorgiaAbstracts.csv')
# files.download("GeorgiaAbstracts.csv")

In [11]:
# a = pd.read_csv('GeorgiaPubmedAbstracts.csv')
# a.columns.values

In [74]:
# Pymed

pubmed = PubMed(tool="MyTool",
                email="bkkashyapbhavya97@gmail.com",
                )
# Georgia Universities
# uni = ['Emory University', 'Morehouse School of Medicine', 'Mercer University', 'Augusta University', 'Pacific Northwest University of Health Sciences College of Osteopathic Medicine']
# Alabama Universities
# uni = ['University of Alabama', 'University of South Alabama', 'Alabama College of Osteopathic Medicine', 'William Carey University College of Osteopathic Med', 'Edward Via College of Osteopathic Medicine', ]
# Alaska Universities
# uni = ['University of Alaska Anchorage', 'Alaska Pacific University', 'University of Alaska Fairbanks', 'University of Alaska Southeast', 'Ilisagvik College']
# Arizona universities
# uni = ['University of Arizona', 'Mayo Clinic Alix School of Medicine']
# Arkansas universities
# uni = ['University of Arkansas', 'Arkansas College of Osteopathic Medicine', 'New York Institute of Technology College of Osteopathic Medicine at Arkansas State University', '']
# California universities
# uni = ['Stanford', 'University of California', 'University of South California', 'Touro University of California', 'Western University of Health Sciences', 
#         'California Health Sciences University College of Osteopathic Medicine', 'California Northstate University', 'California University of Science and Medicine',
#        'Kaiser Permanente School of Medicine', 'Loma Linda University', 'Charles Drew', 'UCLA']
# Colorado Universities
# uni = ['University of Colorado', 'Rocky Vista University College of Osteopathic Medicine']
# Connecticut universities
# uni = ['Frank H. Netter MD School of Medicine at Quinnipiac University', 'University of Connecticut School of Medicine', 'Yale School of Medicine']
# Delaware universities
# uni = ['University of Delaware', 'Wilmington University', 'Delaware State University', 'Goldey Beacom College', 'Delaware Technical Community College Terry', 'Wesley College']
# District of Colombia universities
# uni = ['George Washington University School of Medicine and Health Sciences', 'Georgetown University School of Medicine', 'Howard University College of Medicine']
# Florida universities
# uni = ['University of Florida', 'University of Miami (Miller)', 'USF Morsani College of Medicine', 'University of Central Florida',
#        'Florida Atlantic University (Schmidt)', 'Florida State University', 'Florida International University Herbert Wertheim College of Medicine',
#        'Nova Southeastern University Dr. Kiran C. Patel Allopathic College of Medicine']
# Hawaii universities
# uni = ['University of Hawaii']
# Idaho universities
# uni = ['Idaho College of Osteopathic Medicine']
# Illinois universities
# uni = ['Carle Illinois College of Medicine', 'Chicago Medical School at Rosalind Franklin University of Medicine & Science', 'Loyola University Chicago Stritch School of Medicine',
#        'Northwestern University The Feinberg School of Medicine', 'Rush Medical College of Rush University Medical Center', 'Southern Illinois University School of Medicine',
#        'University of Chicago Division of the Biological Sciences The Pritzker School of Medicine', 'University of Illinois College of Medicine']
# Indiana universities
# uni = ['Indiana University School of Medicine', 'Marian University College of Osteopathic Medicine']
# Iowa universities
# uni = ['University of Iowa']
# Kansas universities
# uni = ['University of Kansas School of Medicine']
# Kentucky universities
# uni = ['University of Kentucky College of Medicine', 'University of Louisville School of Medicine', 'University of Pikeville Kentucky College of Osteopathic Medicine']
# Louisiana university
# uni = ['Louisiana State University School of Medicine', 'Louisiana State University School of Medicine', 'Tulane University School of Medicine']
# Maine universities
# uni = ['University of New England College of Osteopathic Medicine']
# Maryland universities
# uni = ['Johns Hopkins University School of Medicine', 'Uniformed Services University of the Health Sciences F. Edward Hebert School of Medicine', 'University of Maryland School of Medicine']
# Massachusettes universities
# uni = ['Boston University School of Medicine', 'Harvard Medical School', 'Tufts University School of Medicine', 'University of Massachusetts Medical School (Worcester)']
# Michigan universities
# uni = ['Central Michigan University College of Medicine', 'Michigan State University College of Human Medicine', 'Oakland University William Beaumont School of Medicine',
#        'University of Michigan Medical School', 'Wayne State University School of Medicine', 'Western Michigan University Homer Stryker M.D. School of Medicine']
# Minessota universities
# uni = ['Mayo Clinic Alix School of Medicine', 'University of Minnesota Medical School', 'University of Minnesota Medical School']
# Mississippi universities
# uni = ['University of Mississippi School of Medicine', 'William Carey University College of Osteopathic Medicine']
# Missouri universities
uni = ['Saint Louis University School of Medicine', 'University of Missouri', 'Washington University School of Medicine', 'A.T. Still University Kirksville College of Osteopathic Medicine', 
       'Kansas City University of Medicine and Biosciences College of Osteopathic Medicine']
date = '("2005/01/01"[Date - Publication] : "2014/01/01"[Date - Publication])'
j = pd.read_csv('journals.csv')

uni_query = '('
for i in range(len(uni)):
  uni_query += '(' + uni[i].upper() + '[AFFILIATION]' + ')' + ' OR '
uni_query = uni_query[:-4] + ')' + ' AND ' + date + ' AND ' + '('
 
# initialize a dataframe : 
art = pd.DataFrame()

# query and fetch data for each journal out of the selected 145 journals
for k in range(len(j)):
  print(k, j.Title[k])
  query = uni_query + str(j.Title[k]).upper() + '[JOURNAL])'
  #print(query)
  # no more than 3 requests in a second
  time.sleep(0.5)
  results = list(pubmed.query(query, max_results=50000))
  # article_dict = {}
  article_dict_list = []

  # Loop over the retrieved articles
  for i, article in enumerate(results):
      article_dict = {}
      if not article.abstract:
          pass 
      else:
          article_dict['Abstract'] = article.abstract
          try:
              article_dict["Conclusions"]= article.conclusions
          except:
              article_dict["Conclusions"]= 'None'
          article_dict["Copyrights"] = article.copyrights
          article_dict["DOI"] = article.doi
          try:
              article_dict["Journal"] = article.journal
          except:
              article_dict["Journal"] ='None'
          try:
              article_dict["Methods"] = article.methods
          except:
              article_dict["Methods"]='None'
          article_dict["Publication_date"] = article.publication_date
          article_dict["PubmedID"]= article.pubmed_id
          try:
              article_dict["Results"]=article.results
          except:
              article_dict["Results"]='None'
          article_dict["Title"] = article.title
          try:
              article_dict["Keywords"] = '; '.join(article.keywords) 
          except:
              article_dict["Keywords"] = "None"
          authors = []
          affils = []
          for a,author in enumerate(article.authors):
              # print(article.authors)
              auth = []
              auth.append(author['lastname'])
              auth.append(author['firstname'])
              auth.append(author['initials'])
              author_name = ', '.join(str(x) for x in auth)
              authors.append(author_name)

              try:
                  if author['affiliation']!= None:
                      affils.append(author['affiliation'])
                  else:
                      affils.append('None')
              except:
                  affils.append('None')


          article_dict["Authors"] = '; '.join(authors) 
          article_dict["Affiliations"] = '; '.join(affils) 


          article_dict_list.append(article_dict)
  print(len(article_dict_list), ' results appending to art dataframe')
  art = pd.concat([art, pd.DataFrame(article_dict_list)])


0 MMWR Recommendations and Reports
0  results appending to art dataframe
1 New England Journal of Medicine
17  results appending to art dataframe
2 Nature Medicine
8  results appending to art dataframe
3 Nature Reviews Immunology
15  results appending to art dataframe
4 Morbidity and Mortality Weekly Report
0  results appending to art dataframe
5 Lancet
5  results appending to art dataframe
6 Physiological Reviews
5  results appending to art dataframe
7 Nature Reviews Drug Discovery
0  results appending to art dataframe
8 Annual Review of Pathology: Mechanisms of Disease
0  results appending to art dataframe
9 Journal of Clinical Oncology
31  results appending to art dataframe
10 Lancet Global Health
0  results appending to art dataframe
11 Annals of Oncology
3  results appending to art dataframe
12 Annual Review of Public Health
2  results appending to art dataframe
13 Nature Reviews Disease Primers
0  results appending to art dataframe
14 Accounts of Chemical Research
5  results appe

In [75]:
len(art)

1377

In [76]:
art.to_csv('MissouriPubmedFullRecord.csv', index=False)
files.download('MissouriPubmedFullRecord.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
# art.describe()

In [ ]:
j = pd.read_csv('journals.csv')
s = " "
for i in range(len(j)):
  s = s + str(j.Title[i]) + ', '
s

" MMWR Recommendations and Reports, New England Journal of Medicine, Nature Medicine, Nature Reviews Immunology, Morbidity and Mortality Weekly Report, Lancet, Physiological Reviews, Nature Reviews Drug Discovery, Annual Review of Pathology: Mechanisms of Disease, Journal of Clinical Oncology, Lancet Global Health, Annals of Oncology, Annual Review of Public Health, Nature Reviews Disease Primers, Accounts of Chemical Research, Annual Review of Clinical Psychology, Journal of Experimental Medicine, Molecular Systems Biology, Annual Review of Microbiology, JAMA - Journal of the American Medical Association, Lancet Digital Health, Reports on Progress in Physics, Nature Biomedical Engineering, EMBO Journal, JAMA Psychiatry, Nature Reviews Molecular Cell Biology, Cell, Nature Reviews Genetics, Nature Biotechnology, Nature Reviews Cancer, Nature Genetics, Annual Review of Biochemistry, Nature Catalysis, Nature Methods, Cancer Cell, Molecular Cell, Nature Cell Biology, Lancet Diabetes and En

In [ ]:
# https://www.kaggle.com/code/summerkrankin/pubmed-download-als/notebook